In [41]:
import numpy as np
import pandas as pd
import math
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction import FeatureHasher
import collections
import string
import re
import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
work_dir = %pwd # Get the working directory.
training_set = pd.read_csv(str(work_dir + "/" + "train.csv")) # Read in the training data.
training_set = training_set[0:10]

In [40]:
letter_list = [i for i in string.ascii_lowercase]
train_text_lol = [i.lower().split() for i in training_set['text']]
new_lol = []
for i in train_text_lol:
    sub_lol = []
    for j in i:
        sub_lol.append(j.strip(string.punctuation))
    new_lol.append(sub_lol)
train_text_lol = new_lol


all_words = [j for i in train_text_lol for j in i]
word_counts = collections.Counter(all_words)
unique_words = set([j for i in train_text_lol for j in i])

text_counts = {}
for i in unique_words:
    if i not in text_counts:
        text_counts[i] = 0

In [45]:
word_counts

Counter({'a': 5,
         'abandoned': 1,
         'able': 1,
         'afforded': 1,
         'ago': 1,
         'air': 1,
         'all': 2,
         'an': 2,
         'analogy': 1,
         'and': 11,
         'as': 5,
         'ascertaining': 1,
         'astronomer': 1,
         'at': 2,
         'atomies': 1,
         'attempts': 1,
         'attractions': 1,
         'avoid': 1,
         'aware': 1,
         'be': 2,
         'being': 3,
         'believed': 1,
         'beneath': 1,
         'best': 1,
         'board': 1,
         'body': 1,
         'box': 1,
         'brought': 1,
         'brutality': 1,
         'but': 1,
         'by': 2,
         'cannot': 1,
         'capered': 1,
         'casting': 1,
         'certainly': 1,
         'character': 1,
         'cheering': 1,
         'circuit': 1,
         'code': 1,
         'confess': 1,
         'confirmation': 1,
         'cosmogony': 1,
         'cottages': 1,
         'could': 2,
         'countenance': 1,
      

In [32]:

for i in range(0,len(train_text_lol)):
    count_words = collections.Counter(train_text_lol[i])
    for j in text_counts.keys():
        text_counts[j] = count_words[j]
text_counts

{'a': 0,
 'abandoned': 0,
 'able': 0,
 'afforded': 0,
 'ago': 0,
 'air': 0,
 'all': 0,
 'an': 0,
 'analogy': 0,
 'and': 0,
 'as': 0,
 'ascertaining': 0,
 'astronomer': 0,
 'at': 0,
 'atomies': 0,
 'attempts': 0,
 'attractions': 1,
 'avoid': 0,
 'aware': 0,
 'be': 0,
 'being': 0,
 'believed': 0,
 'beneath': 0,
 'best': 0,
 'board': 0,
 'body': 0,
 'box': 0,
 'brought': 0,
 'brutality': 0,
 'but': 0,
 'by': 0,
 'cannot': 0,
 'capered': 0,
 'casting': 0,
 'certainly': 0,
 'character': 0,
 'cheering': 0,
 'circuit': 0,
 'code': 1,
 'confess': 1,
 'confirmation': 0,
 'cosmogony': 0,
 'cottages': 0,
 'could': 0,
 'countenance': 0,
 'counties': 0,
 'crew': 0,
 'cutting': 0,
 'desk': 0,
 'dimensions': 0,
 'discussed': 0,
 'distaste': 0,
 'do': 0,
 'down': 0,
 'dungeon': 0,
 'else': 0,
 'epicurus': 0,
 'equally': 0,
 'even': 0,
 'exercised': 0,
 'expected': 0,
 'eyes': 0,
 'fact': 0,
 'fair': 0,
 'fall': 0,
 'fantastic': 0,
 'felt': 0,
 'feminine': 0,
 'fertile': 0,
 'finding': 0,
 'for': 1,
 '

In [11]:

counted = dict(collections.Counter(train_text_lol[0]))

c = collections.Counter(train_text_lol[0])

0

In [68]:
hasher = FeatureHasher(input_type='string')
hashed_text = hasher.transform(training_set['text'])
hashed_text_array = hashed_text.toarray()

In [70]:
hasher.get_params()

{'alternate_sign': True,
 'dtype': numpy.float64,
 'input_type': 'string',
 'n_features': 1048576,
 'non_negative': False}

In [57]:
training_set

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL
5,id22965,"A youth passed in solitude, my best years spen...",MWS
6,id09674,"The astronomer, perhaps, at this point, took r...",EAP
7,id13515,The surcingle hung in ribands from my body.,EAP
8,id19322,I knew that you could not say to yourself 'ste...,EAP
9,id00912,I confess that neither the structure of langua...,MWS


In [ ]:
gaussian_nb = GaussianNB()
gaussian_nb.fit(iris.data, iris.target).predict(iris.data)